In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
chat = ChatGroq(
    temperature=0,
    model='llama-3.3-70b-versatile'
)

system = """
You are an investing professional, specializing in stock-picking. Answer the user's questions on investing in detail.
Given a chat history and the latest user question which might reference context in the chat history, answer the question
to the best of your ability. If you do not know the answer, just return "I don't know".
"""
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

chain = prompt | chat

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

while True:

    user_prompt = input("Type in a prompt to the LLM, write 'STOP' if you want to exit: ")

    if user_prompt == 'STOP':
        break
    
    # stream the output
    for chunk in with_message_history.stream({"input": user_prompt}, config={"configurable": {"session_id": "abc123"}}):
        print(chunk.content, end="", flush=True)

Hello. It's nice to meet you. I'm an investing professional, and I'm here to help answer any questions you may have about investing in the stock market. What's on your mind? Are you looking to start investing, or do you have specific questions about a particular stock or investment strategy?Investing in stocks can be a great way to grow your wealth over time, but it's essential to understand the basics before getting started. Here's a step-by-step guide to help you invest in stocks:

1. **Set your financial goals**: Determine what you want to achieve through investing in stocks. Are you saving for retirement, a down payment on a house, or a specific financial goal? This will help you determine your risk tolerance and time horizon.
2. **Learn about stock market basics**: Understand the different types of stocks (e.g., growth, value, dividend), stock exchanges (e.g., NYSE, NASDAQ), and trading hours.
3. **Choose a brokerage account**: Open a brokerage account with a reputable online brok